In [1]:
%load_ext autoreload
%autoreload 2

# data core

> data core structures for eos

In [2]:
#| default_exp data.core

In [3]:
#| export
import re
from configparser import ConfigParser
from datetime import datetime
from pathlib import Path
from typing import Dict, List, NamedTuple, Optional, Tuple, TypeVar, Union, TypeAlias
from zoneinfo import ZoneInfo  # type: ignore


In [4]:
#| export
import pandas as pd  # type: ignore
from pydantic import BaseModel, Field, GetPydanticSchema  # type: ignore
from pydantic.dataclasses import dataclass  # type: ignore
from typing_extensions import TypedDict
from data_io_nbdev.data.location import EosLocation


In [5]:
#| hide
from nbdev.showdoc import *

In [6]:
#| export
KvaserType: TypeAlias = dict[str, str]
RCANType: TypeAlias = dict[str, dict[str, list[Union[str, list[list[str]]]]]]
RawType: TypeAlias = Union[KvaserType, RCANType]


In [7]:
#| export
#  Define TypedDict for type hinting of typed collections: records and episodes
veos_lifetime_start_date: pd.Timestamp = pd.Timestamp(
    ts_input="2021-01-01T00:00:00+08:00", tz="Asia/Shanghai"
)
veos_lifetime_end_date: pd.Timestamp = pd.Timestamp(
    ts_input="2031-12-31T00:00:00+08:00", tz="Asia/Shanghai"
)


In [8]:
#| export
class MotionPower(NamedTuple):
    """
    Motion power tuple
    
    Attributes:
        
        timestep: timestamp of the tuple
        velocity: velocity in m/s,
        thrust: thrust in percentage of full acc pedal,
        brake: brake in percentage of full brake pedal,
        current: current in A,
        voltage: voltage in V,
    
    return:
    
            MotionPower tuple    
    """
    timestep: pd.Timestamp  # timestamp of the tuple
    velocity: float  # velocity in m/s
    thrust: float  # thrust in percentage of full acc pedal
    brake: float  # brake in percentage of full brake pedal
    current: float  # current in A
    voltage: float  # voltage in V


In [9]:
show_doc(MotionPower)

---

[source](https://github.com/Binjian/data-io-nbdev/tree/main/blob/main/data_io_nbdev/data/core.py#L41){target="_blank" style="float:right; font-size:smaller"}

### MotionPower

>      MotionPower (timestep:pandas._libs.tslibs.timestamps.Timestamp,
>                   velocity:float, thrust:float, brake:float, current:float,
>                   voltage:float)

Motion power tuple

Attributes:

    timestep: timestamp of the tuple
    velocity: velocity in m/s,
    thrust: thrust in percentage of full acc pedal,
    brake: brake in percentage of full brake pedal,
    current: current in A,
    voltage: voltage in V,

return:

        MotionPower tuple

In [10]:
#| export
class ECUMixin(BaseModel):
    """
    ECU mixin class
    
    Attributes:
        
        ecu_observation_number: number of observations per ECU
        ecu_observation_frequency: frequency of observations per ECU
        ecu_countdown: countdown time for ECU
        
    return:
    
            ECU mixin class
    """
    # optional: can be adjusted by developer
    ecu_observation_number: int = 30
    #     (KvaserMixin.kvaser_observation_number
    # )  # 30  # Kvaser number of one observation unit: 30 as count number
    ecu_observation_frequency: int = 20
    #     (KvaserMixin.kvaser_observation_frequency
    #     # 20  # Kvaser observation frequency: 20 Hz, fixed by hardware setting
    # )
    ecu_countdown: int = 3
    #     (KvaserMixin.kvaser_countdown
    #     # 3  # Kvaser countdown time: 3 seconds, optional: can be adjusted by developer
    # )

In [11]:
show_doc(ECUMixin)

---

[source](https://github.com/Binjian/data-io-nbdev/tree/main/blob/main/data_io_nbdev/data/core.py#L67){target="_blank" style="float:right; font-size:smaller"}

### ECUMixin

>      ECUMixin (ecu_observation_number:int=30,
>                ecu_observation_frequency:int=20, ecu_countdown:int=3)

ECU mixin class

Attributes:

    ecu_observation_number: number of observations per ECU
    ecu_observation_frequency: frequency of observations per ECU
    ecu_countdown: countdown time for ECU

return:

        ECU mixin class

In [12]:
#| exports
def say_hi(to: str):
    """
    Say hello to somebody
    
    
    Args:
        to: somebody to say hello to
    """
    
    return f'Hi {to}!'

In [13]:
from fastcore.test import *

In [14]:
test_eq(say_hi("Sylvain"), "Hi Sylvain!")

In [15]:
#| hide
import nbdev; nbdev.nbdev_export()
